In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


# !kaggle datasets download -d hijest/cleaned-data-for-the-chatbot-collected-from-movies

# !unzip /content/cleaned-data-for-the-chatbot-collected-from-movies.zip

# !rm -rf /content/cleaned-data-for-the-chatbot-collected-from-movies.zip

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('Conversation.csv',index_col='Unnamed: 0')
df.head()

wordArr = df['question'].to_list()
wordArr_clean = [ re.sub(r'[^\w\s]', '', x).lower() for x in wordArr if x != '' and type(x) == str]
wordArr_clean


['hi how are you doing',
 'im fine how about yourself',
 'im pretty good thanks for asking',
 'no problem so how have you been',
 'ive been great what about you',
 'ive been good im in school right now',
 'what school do you go to',
 'i go to pcc',
 'do you like it there',
 'its okay its a really big campus',
 'good luck with school',
 'hows it going',
 'im doing well how about you',
 'never better thanks',
 'so how have you been lately',
 'ive actually been pretty good you',
 'im actually in school right now',
 'which school do you attend',
 'im attending pcc right now',
 'are you enjoying it there',
 'its not bad there are a lot of people there',
 'good luck with that',
 'how are you doing today',
 'im doing great what about you',
 'im absolutely lovely thank you',
 'everythings been good with you',
 'i havent been better how about yourself',
 'i started school recently',
 'where are you going to school',
 'im going to pcc',
 'how do you like it so far',
 'i like it so far my classes

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(wordArr_clean)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in wordArr_clean:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)
input_sequences

[[1000, 36],
 [1000, 36, 14],
 [1000, 36, 14, 2],
 [1000, 36, 14, 2, 155],
 [25, 623],
 [25, 623, 36],
 [25, 623, 36, 33],
 [25, 623, 36, 33, 537],
 [25, 147],
 [25, 147, 44],
 [25, 147, 44, 284],
 [25, 147, 44, 284, 24],
 [25, 147, 44, 284, 24, 474],
 [29, 171],
 [29, 171, 21],
 [29, 171, 21, 36],
 [29, 171, 21, 36, 13],
 [29, 171, 21, 36, 13, 2],
 [29, 171, 21, 36, 13, 2, 96],
 [94, 96],
 [94, 96, 103],
 [94, 96, 103, 9],
 [94, 96, 103, 9, 33],
 [94, 96, 103, 9, 33, 2],
 [94, 96],
 [94, 96, 44],
 [94, 96, 44, 25],
 [94, 96, 44, 25, 15],
 [94, 96, 44, 25, 15, 80],
 [94, 96, 44, 25, 15, 80, 72],
 [94, 96, 44, 25, 15, 80, 72, 120],
 [9, 80],
 [9, 80, 8],
 [9, 80, 8, 2],
 [9, 80, 8, 2, 37],
 [9, 80, 8, 2, 37, 4],
 [1, 37],
 [1, 37, 4],
 [1, 37, 4, 749],
 [8, 2],
 [8, 2, 19],
 [8, 2, 19, 6],
 [8, 2, 19, 6, 54],
 [18, 100],
 [18, 100, 18],
 [18, 100, 18, 5],
 [18, 100, 18, 5, 48],
 [18, 100, 18, 5, 48, 142],
 [18, 100, 18, 5, 48, 142, 1436],
 [44, 1001],
 [44, 1001, 46],
 [44, 1001, 46, 80

In [ ]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])



model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
623/623 [==============================] - 11s 15ms/step - loss: 6.3634 - accuracy: 0.0373
Epoch 2/500
623/623 [==============================] - 10s 16ms/step - loss: 5.9671 - accuracy: 0.0539
Epoch 3/500
623/623 [==============================] - 10s 16ms/step - loss: 5.7286 - accuracy: 0.0748
Epoch 4/500
623/623 [==============================] - 10s 16ms/step - loss: 5.5279 - accuracy: 0.0818
Epoch 5/500
623/623 [==============================] - 10s 16ms/step - loss: 5.3588 - accuracy: 0.0883
Epoch 6/500
623/623 [==============================] - 10s 16ms/step - loss: 5.1967 - accuracy: 0.0971
Epoch 7/500
623/623 [==============================] - 10s 16ms/step - loss: 5.0318 - accuracy: 0.1084
Epoch 8/500
623/623 [==============================] - 10s 16ms/step - loss: 4.8652 - accuracy: 0.1177
Epoch 9/500
623/623 [==============================] - 10s 16ms/step - loss: 4.6998 - accuracy: 0.1328
Epoch 10/500
623/623 [==============================] - 10s 16ms/step - l

In [ ]:
# Generate next word predictions
seed_text = "there are a lot of people "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 24ms/step
Next predicted words: there are a lot of people  yesterday a can perfume the
